**import libraries**



In [1]:
%pip install pyproj
%pip install shapely
%pip install folium

In [2]:
import pandas as pd
import numpy as np
import datetime
import pyproj
from shapely.geometry import Polygon, Point
from shapely.vectorized import contains as CN
import folium
import branca.colormap as cm

**Define required functions**


In [3]:
def read_rawdata(file_name, AoI):
    """
    it downloads rawdata in zip format, unzip the file, and filter out AIS records located in AoI.
    
    Argument:
    file_name -- a string representing download link where the zip file located
    AoI -- array of double =[[lower bound latitude, upper bound latitude], [lower bound longitude, upper bound longitude]]
    
    Returns:
    clean_data -- AIS data in array
    """
    print(datetime.datetime.now().strftime("%H:%M:%S")+ " : unziping " + file_name.split("/")[-1] + "...")
    df = pd.read_csv(file_name,compression='zip')

    print(datetime.datetime.now().strftime("%H:%M:%S")+ " : cleaning " +  file_name.split("/")[-1] + "...")
    intersect = (df.iloc[:,2].values >= AoI[0][0])*(df.iloc[:,2].values <= AoI[0][1])*(df.iloc[:,3].values >= AoI[1][0])*(df.iloc[:,3].values <= AoI[1][1])
    data_in_HSC = df.iloc[:,:].values
    data_in_HSC = data_in_HSC[intersect]
    
    #clear NA data
    na_mmsi = 1-np.isnan(data_in_HSC[:,0].astype(np.float64))
    na_date = 1-np.isnat(pd.to_datetime(data_in_HSC[:,1]))
    na_log = 1-np.isnan(data_in_HSC[:,3].astype(np.float64))
    na_lat = 1-np.isnan(data_in_HSC[:,2].astype(np.float64))
    na_speed = 1-np.isnan(data_in_HSC[:,4].astype(np.float64))
    na_vesseltype = 1-(data_in_HSC[:,10].astype(np.str)=='nan')

    is_valid = na_mmsi*na_date*na_log*na_lat*na_speed*na_vesseltype
    clean_data = data_in_HSC[(is_valid==1)]
    
    return clean_data
  
def time_to_num(date, unit = 's', origin = '1970-01-01T00:00:00'):
    """
    it converts datetime into number
    
    Argument:
    date -- array of datetime in string or datetime format
    unit -- unit of time in string 'Y': year, 'M': month, 'D': day, 'h': hour, 'm':minute, 's':second
    origin -- the origin datetime predefined value is 1970-01-01T00:00:00

    Returns:
    array of datetime in numbers
    """ 
    num = (date.astype(np.datetime64) - np.datetime64(origin)) / np.timedelta64(1, unit)
    return np.reshape(num, newshape=(len(num),1))

def num_to_time(num, unit = 's', origin = '1970-01-01T00:00:00'):
    """
    it converts number into datetime

    Argument:
    num -- array of datetime in number
    unit -- unit of time in string 'Y': year, 'M': month, 'D': day, 'h': hour, 'm':minute, 's':second
    origin -- the origin datetime predefined value is 1970-01-01T00:00:00

    Returns:
    array of dates in datetime format
    """ 
    date = num*np.timedelta64(1, unit) +np.datetime64(origin)
    return np.reshape(date, newshape=(len(date),1))

def time_to_steps(date, sample_rate, unit = 's', origin = '1970-01-01T00:00:00'):
    """
    it converts datetime into timestamp based on a given sample rate and unit

    Argument:
    date -- array of datetime in string or datetime
    sample_rate -- integer number representing the given sample rate
    unit -- unit of the sample rate in string 'Y': year, 'M': month, 'D': day, 'h': hour, 'm':minute, 's':second
    origin -- the origin datetime predefined value is 1970-01-01T00:00:00

    Returns:
    steps -- array of timestamps in numbers
    """ 
    
    steps = sample_rate*np.floor(((date - np.datetime64(origin)) / np.timedelta64(1, unit))/sample_rate)
    return np.reshape(steps, newshape=(len(steps),1))

def time_to_interval(date, interval_length = 1, unit = 'h'):
    """
    it converts datetime into date and time intervals based on a given interval length and unit

    Argument:
    date -- array of datetime in string or datetime
    interval_length -- integer number representing the given  interval length 
    unit -- unit of the interval length in string 'Y': year, 'M': month, 'D': day, 'h': hour, 'm':minute, 's':second

    Returns:
    date_interval -- array of date intervals in string format
    t_interval -- array of time intervals in numbers

    """ 
    date_interval = date.astype('datetime64[D]')
    t_interval = interval_length*np.floor((date.astype(np.datetime64) - date_interval) / np.timedelta64(interval_length, unit))
    
    return date_interval.astype(np.str), t_interval

def point_in_polygon(segments, x, y):
    """
    it gets the intersect of all coordinations and the given segments as an array which show the segment number of each point.
    
    Argument:
    segments -- array of shape (m,1) representing an array of objects of m segments defined in AoI in which each object has {index, 'name', and 'points'} 
    x -- array of shape (n) representing latitudes of n points
    y -- array of shape (n) representing longitudes of n points

    Returns:
    segs -- array of shape(n) representing segment index of n points
    """ 
    intersect = []
    for seg in segments:
        cords = seg['points']
        seg = Polygon(cords)
        i = CN(seg, x,y)
        intersect.append(i)
        
    segs = np.argmax(intersect,axis=0)+1
    segs[np.sum(intersect,axis=0)==0] = -1

    return segs

def pairwise_subtract(array, columns = [0], stride=1, forward = True):
    """
    it takes an array and applies subtraction to all pairs of consecutive points.
    
    Argument:
    array -- array of shape (n_row,n_col) representing input value 
    columns -- array of column index(s) to which we want to apply subtraction
    stride -- integer number representing forward or backward steps to create pair points
    forward -- boolean variable determining if the stride step is forward or backward

    Returns:
    sub -- array of shape(n_row, n_col2) representing subtraction of consecutive points in which n_col2 is the shape of columns
    """ 
 
    sub = np.zeros((len(array), len(columns)))
    if forward:        
        sub[stride:,:] = array[stride:, columns] - array[:-stride, columns]
    else:
        sub[stride:,:] = array[:-stride, columns] - array[stride:, columns]
        
    return sub

def pairwise_distance(x, y):
    """
    it takes an array and latitudes and logitudes and calculate spatial distance between each consecutive points.
    
    Argument:
    x -- array of shape (n) representing latitudes of n points
    y -- array of shape (n) representing longitudes of n points

    Returns:
    dis -- array of shape(n) representing spatial distance between consecutive points
    """ 
    projector = pyproj.Proj(proj='utm',zone=15,datum='NAD83', ellps='WGS84')
    new_y,new_x = projector(y, x)
    new_xy = np.column_stack((new_x,new_y))
    dis = pairwise_subtract(new_xy, [0,1])
    dis = (dis**2).sum(axis=1, keepdims=True)**0.5
    
    return dis

def pairwise_if(array, columns = [0], stride=1, forward = True):
    """
    it takes an array and check if two consecutive points are equal.
    
    Argument:
    array -- array of shape (n_row,n_col) representing input value 
    columns -- array of column index(s) to which we want to apply subtraction
    stride -- integer number representing forward or backward steps to create pair points
    forward -- boolean variable determining if the stride step is forward or backward

    Returns:
    if_equal -- array of shape(n_row) representing if consecutive points are equal:1 otherwise:0
    """ 
    if_equal = np.zeros(len(array))
    con = (array[stride:, columns] == array[:stride, columns])
    if forward: 
        con = (array[stride:, columns] == array[:-stride, columns])

    if_equal[stride:] = np.prod(con, axis = 1)
    return if_equal

**Step 0: Load Data and define required variables**

Load AIS data of August 2018 are download from https://marinecadastre.gov/ais/ and after cleaning are merged into one file.
the raw data has following columns:
* 0 : MMSI
* 1 : BaseDateTime
* 2 : LAT
* 3 : LON
* 4 : SOG
* 5 : COG
* 6 : Heading
* 7 : VesselName
* 8 : IMO
* 9 : CallSign
* 10 : VesselType
* 11 : Status
* 12 : Length
* 13 : Width
* 14 : Draft
* 15 : Cargo
* 16 : TranscieverClass


AoI is defined to get Houston Ship Channel Data

AoI is divided inot Segments and lines

In [5]:
#create AIS files link
start_date = "2018-08-01"
end_date = "2018-08-31"

file_names =[]
file_dir = 'https://coast.noaa.gov/htdata/CMSP/AISDataHandler'
for dates in np.array(pd.date_range(start=start_date,end=end_date)).astype('datetime64[D]'):
  file_names.append("%s/%s/AIS_%s.zip"%(file_dir,str(dates)[:4],str(dates).replace("-","_")))

#filter Houston Data
AoI = [[28.953797, 29.8327499],[-95.4448031, -93.837573]]

#load segments polygon
path = 'segments.txt'
f = open(path, "r")
raw_seg = f.readlines()
segments = []

for r in raw_seg:
    d = r.replace(',',"").split()
    name = d[0]
    n = len(d) if len(d)%2==1 else len(d)-1
    points = np.array(d[1:n], dtype= 'float64').ravel()
    points = np.reshape(points, (int(n/2),2))    
    segments.append({"name": name, "points": points})
    
path = 'lines.txt'
f = open(path, "r")
raw_seg = f.readlines()
segments_points = []

for r in raw_seg:
    d = r.replace(',',"").split()
    name = d[0]
    n = len(d) if len(d)%2==1 else len(d)-1
    points = np.array(d[1:n], dtype= 'float64').ravel()
    points = np.reshape(points, (int(n/2),2))    
    segments_points.append({"name": name, "points": points})

#load and stack files
for i in range(len(file_names)):
    clead_data = read_rawdata(file_names[i], AoI)
    if i==0:
      raw_data = clead_data
    else:
      raw_data = np.vstack((raw_data,clead_data))

23:06:50 : unziping AIS_2018_08_01.zip...
23:07:12 : cleaning AIS_2018_08_01.zip...


<ipython-input-3-0e5875e0ec1e>:26: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  na_vesseltype = 1-(data_in_HSC[:,10].astype(np.str)=='nan')


23:07:16 : unziping AIS_2018_08_02.zip...
23:07:39 : cleaning AIS_2018_08_02.zip...
23:07:44 : unziping AIS_2018_08_03.zip...
23:08:06 : cleaning AIS_2018_08_03.zip...
23:08:13 : unziping AIS_2018_08_04.zip...
23:08:38 : cleaning AIS_2018_08_04.zip...
23:08:43 : unziping AIS_2018_08_05.zip...
23:09:06 : cleaning AIS_2018_08_05.zip...
23:09:11 : unziping AIS_2018_08_06.zip...
23:09:36 : cleaning AIS_2018_08_06.zip...
23:09:43 : unziping AIS_2018_08_07.zip...
23:10:15 : cleaning AIS_2018_08_07.zip...
23:10:21 : unziping AIS_2018_08_08.zip...
23:10:52 : cleaning AIS_2018_08_08.zip...
23:11:01 : unziping AIS_2018_08_09.zip...
23:11:24 : cleaning AIS_2018_08_09.zip...
23:11:33 : unziping AIS_2018_08_10.zip...
23:12:01 : cleaning AIS_2018_08_10.zip...
23:12:17 : unziping AIS_2018_08_11.zip...


c:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3357: DtypeWarning: Columns (7,8,9) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


23:12:37 : cleaning AIS_2018_08_11.zip...
23:12:44 : unziping AIS_2018_08_12.zip...
23:13:04 : cleaning AIS_2018_08_12.zip...
23:13:12 : unziping AIS_2018_08_13.zip...
23:13:34 : cleaning AIS_2018_08_13.zip...
23:13:45 : unziping AIS_2018_08_14.zip...
23:14:08 : cleaning AIS_2018_08_14.zip...
23:14:17 : unziping AIS_2018_08_15.zip...
23:14:38 : cleaning AIS_2018_08_15.zip...
23:14:46 : unziping AIS_2018_08_16.zip...
23:15:08 : cleaning AIS_2018_08_16.zip...
23:15:18 : unziping AIS_2018_08_17.zip...
23:15:44 : cleaning AIS_2018_08_17.zip...
23:15:55 : unziping AIS_2018_08_18.zip...
23:16:24 : cleaning AIS_2018_08_18.zip...
23:16:35 : unziping AIS_2018_08_19.zip...
23:16:58 : cleaning AIS_2018_08_19.zip...
23:17:10 : unziping AIS_2018_08_20.zip...
23:17:36 : cleaning AIS_2018_08_20.zip...
23:17:56 : unziping AIS_2018_08_21.zip...
23:18:21 : cleaning AIS_2018_08_21.zip...
23:18:42 : unziping AIS_2018_08_22.zip...
23:19:07 : cleaning AIS_2018_08_22.zip...
23:19:32 : unziping AIS_2018_08_23

**Step 1: Apply sample rate of 5 minutes**

In [6]:
#applying Sample rate
date_col= raw_data[:,1].astype(np.datetime64) # datetime columns
steps = time_to_steps(date_col.copy(),5,unit = 'm')

raw_data = np.column_stack(( raw_data,steps))
_,indices = np.unique(raw_data[:,[0,-1]].astype(np.float64), axis=0, return_index=True)

data_in_steps = raw_data[indices]


**Step 2: Find segments of each AIS records**

In [7]:
#find intersect
x = np.array(data_in_steps[:,2],dtype = "float64")
y = np.array(data_in_steps[:,3],dtype = "float64")
intersect = point_in_polygon(segments, x, y)

data_in_segment = np.column_stack((data_in_steps,intersect))

data_in_segment = data_in_segment[(intersect>0),:]


**Step 3.0: Sort data based on Date and MMSI**

In [8]:
#sorting  based on Date/Time and MMSI

data_in_order = data_in_segment[np.lexsort((data_in_segment[:,1], data_in_segment[:,0]))]


**Step 3.1: Find Direction of each AIS record***

In [9]:
#find direction

#define a bench mark
base_point = np.array([29.48227261, -93.99233543], dtype = "float64")

dis_to_base = ((data_in_order[:,[2,3]]-base_point)**2).sum(axis=1, keepdims= True)

direction = (pairwise_subtract(dis_to_base)>=0).astype(np.int64)
direction[(data_in_order[:,4]<2)] = -1  #stopped vessels

data_in_direction = np.column_stack((data_in_order,direction))


**Step 4.0: Calculate time and distance difference between consecutive points***

In [10]:
#time & distance difference calculation
time_col =time_to_num(data_in_direction[:,1])
delta_time = pairwise_subtract(time_col)

x = np.array(data_in_direction[:,2],dtype = "float64")
y = np.array(data_in_direction[:,3],dtype = "float64")

delta_lenght = pairwise_distance(x,y)


**Step 4.1: Clear noises comparing the real speed and instant speed***

In [11]:
#clear noises
speed = np.nan_to_num(delta_lenght/delta_time*(3.6/1.852)).T.ravel()
is_speed =  1-((abs(speed-data_in_direction[:,4]) <2.5)*(data_in_direction[:,4] < 25))

is_mmsi = pairwise_if(data_in_direction, [0])
is_noise = (1-is_speed*is_mmsi)

cleaned_data = data_in_direction[(is_noise==1)]

time_col =time_to_num(cleaned_data[:,1])
delta_time = pairwise_subtract(time_col)

x = np.array(cleaned_data[:,2],dtype = "float64")
y = np.array(cleaned_data[:,3],dtype = "float64")
delta_lenght = pairwise_distance(x,y)

speed = np.nan_to_num(delta_lenght/delta_time*(3.6/1.852))

data_with_delta = np.column_stack((cleaned_data,delta_time, delta_lenght,speed))

<ipython-input-11-4bd7414b5505>:2: RuntimeWarning: invalid value encountered in true_divide
  speed = np.nan_to_num(delta_lenght/delta_time*(3.6/1.852)).T.ravel()
<ipython-input-11-4bd7414b5505>:17: RuntimeWarning: invalid value encountered in true_divide
  speed = np.nan_to_num(delta_lenght/delta_time*(3.6/1.852))


**Step 5: Spearating trips and tours***

In [12]:
#trip separation

is_trip = pairwise_if(data_with_delta, [0,19])
is_deltaT = (data_with_delta[:,20]>0)*(data_with_delta[:,20]<2*60*60).astype(np.int)

is_trip = is_trip*is_deltaT
trip = np.cumsum(1- is_trip)


is_tour = pairwise_if(data_with_delta, [0])*is_deltaT
tour = np.cumsum(1- is_tour)

data_all_in_one = np.column_stack((data_with_delta,tour, trip))

#Remove redundant recors
data_all_in_one = data_all_in_one[(is_trip==1)]


<ipython-input-12-058107f81bf3>:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  is_deltaT = (data_with_delta[:,20]>0)*(data_with_delta[:,20]<2*60*60).astype(np.int)


**Step 6: Add more columns to the AIS data***

In [13]:
# add columns to data

date_interval,  hour_interval = time_to_interval(data_all_in_one[:,1], 1, 'h')

data_all_in_one[(data_all_in_one[:,19]==-1),19] = "Stopped"
data_all_in_one[(data_all_in_one[:,19]==0),19] = "Inbound"
data_all_in_one[(data_all_in_one[:,19]==1),19] = "Outbound"

length_class = np.floor((np.nan_to_num(data_all_in_one[:,12])/100.0001).astype(np.float64))*100+50

width_class = np.full(shape = (length_class.shape), fill_value = "[0-18)m")
width_class[(np.nan_to_num(data_all_in_one[:,13])>=18)*(np.nan_to_num(data_all_in_one[:,13])<36),] = "[18-36)m"
width_class[(np.nan_to_num(data_all_in_one[:,13])>=36),] = "[36-~)m"

    
draft_class = np.full(shape = (length_class.shape), fill_value = "[0-5)m")
draft_class[(np.nan_to_num(data_all_in_one[:,14])>=5),] = "[5-~)m"

data_all_in_one = np.column_stack((data_all_in_one,date_interval,  hour_interval, length_class, width_class, draft_class))


<ipython-input-3-0e5875e0ec1e>:97: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return date_interval.astype(np.str), t_interval
<ipython-input-13-9f6d5ed0b591>:12: RuntimeWarning: invalid value encountered in greater_equal
  width_class[(np.nan_to_num(data_all_in_one[:,13])>=18)*(np.nan_to_num(data_all_in_one[:,13])<36),] = "[18-36)m"
<ipython-input-13-9f6d5ed0b591>:12: RuntimeWarning: invalid value encountered in less
  width_class[(np.nan_to_num(data_all_in_one[:,13])>=18)*(np.nan_to_num(data_all_in_one[:,13])<36),] = "[18-36)m"
<ipython-input-13-9f6d5ed0b591>:13: RuntimeWarning: invalid value encountered in greater_equal
  width_class[(np.nan_to_num(data_all_in_one[:

**Step 7: Covnert data into dataframe and add column names**

In [14]:
#convert to dataframe
data_df = pd.DataFrame(data_all_in_one)

data_df.columns = ['MMSI' , 'Date/Time' ,'Latitude' , 'Longitude' , 'Speed' , 'COG', 'Heading' , 'Vessel Name' , 'IMO' ,
                   'CallSign' , 'Vessel_Type' , 'NavigationStatus', 'Length' , 'Breadth' , 'MaxDraught' , 'Cargo' , 'TranscieverClass' ,
                   'steps' , 'segment' , 'direction' , 'delta t' , 'delta l' , 'Ave_Speed' , 'tour' , 
                   'trip' , 'date_interval' , 'hour_interval' , 'length_class' , 'width_class', 'draft_class']
data_df

,MMSI,Date/Time,Latitude,Longitude,Speed,COG,Heading,Vessel Name,IMO,CallSign,...,delta t,delta l,Ave_Speed,tour,trip,date_interval,hour_interval,length_class,width_class,draft_class
0,205517000,2018-08-26T23:26:04,29.15828,-94.46158,4.2,250.0,250.0,CAP FELIX,IMO9380738,ONFH,...,279.0,666.073072,4.640654,1.0,1.0,2018-08-26,23.0,250.0,[36-~)m,[5-~)m
1,205517000,2018-08-26T23:32:24,29.1557,-94.47047,5.1,251.0,250.0,CAP FELIX,IMO9380738,ONFH,...,380.0,910.831217,4.659248,1.0,1.0,2018-08-26,23.0,250.0,[36-~)m,[5-~)m
2,205517000,2018-08-26T23:39:24,29.15292,-94.48007,4.0,251.0,243.0,CAP FELIX,IMO9380738,ONFH,...,420.0,983.390564,4.551329,1.0,1.0,2018-08-26,23.0,250.0,[36-~)m,[5-~)m
3,205517000,2018-08-26T23:42:24,29.15162,-94.48328,3.5,243.0,240.0,CAP FELIX,IMO9380738,ONFH,...,180.0,343.905883,3.713886,1.0,1.0,2018-08-26,23.0,250.0,[36-~)m,[5-~)m
4,205517000,2018-08-26T23:48:54,29.14903,-94.48922,2.8,241.0,223.0,CAP FELIX,IMO9380738,ONFH,...,390.0,645.224746,3.21594,1.0,1.0,2018-08-26,23.0,250.0,[36-~)m,[5-~)m
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2759930,866998080,2018-08-20T23:22:03,29.36597,-94.79603,2.3,127.8,511.0,SEA DRIFT,IMO8635904,WDC2818,...,210.0,272.621215,2.523492,8299.0,108466.0,2018-08-20,23.0,50.0,[0-18)m,[0-5)m
2759931,866998080,2018-08-20T23:27:54,29.36378,-94.79442,2.0,102.5,511.0,SEA DRIFT,IMO8635904,WDC2818,...,351.0,288.716035,1.598915,8299.0,108466.0,2018-08-20,23.0,50.0,[0-18)m,[0-5)m
2759932,866998080,2018-08-20T23:43:14,29.36703,-94.7894,4.3,36.3,511.0,SEA DRIFT,IMO8635904,WDC2818,...,221.0,417.285909,3.670312,8299.0,108468.0,2018-08-20,23.0,50.0,[0-18)m,[0-5)m
2759933,866998080,2018-08-20T23:47:54,29.3714,-94.78478,4.2,37.4,511.0,SEA DRIFT,IMO8635904,WDC2818,...,280.0,660.153673,4.582986,8299.0,108468.0,2018-08-20,23.0,50.0,[0-18)m,[0-5)m


**Results**
* results 1: base map

In [15]:
mymap = folium.Map(width=700,height=550, location=[29.512201, -94.885110], zoom_start=10)

for seg in segments:
  label = seg['name']
  points = seg['points']
  folium.Polygon(points, tooltip=label, fill=True, stroke= False, fill_color = 'gray').add_to(mymap)

for seg in segments_points:
  label = seg['name']
  points = seg['points']
  folium.PolyLine(points, tooltip=label).add_to(mymap)

display(mymap)

**Result 1: OD matrix**
* inbound OD matrix and map

In [16]:
OD_data = data_df.groupby(['MMSI', 'direction', 'trip', 'Vessel_Type', 'length_class' , 'width_class']).agg(
    origin = ('segment', lambda x: list(x)[0]), destination =('segment', lambda x: list(x)[-1]), 
    start_time = ('Date/Time', lambda x: list(x)[0]), end_time =  ('Date/Time', lambda x: list(x)[-1]), 
    travel_time = ('delta t', np.sum), travel_length = ('delta l', np.sum))

seg_OD = OD_data.groupby(['direction','origin', 'destination']).agg(demand =('origin', np.count_nonzero)).reset_index()

print(seg_OD)

mymap = folium.Map(width=700,height=550, location=[29.512201, -94.885110], zoom_start=10)

inbound_OD = seg_OD.query('direction=="Inbound" & origin != destination')

colormap = cm.linear.Reds_06.scale(vmin = inbound_OD.demand.min(), vmax = inbound_OD.demand.max())

inbound_OD = inbound_OD.sort_values(by='demand')
for i in inbound_OD.index:
  o_id = inbound_OD['origin'][i]
  d_id = inbound_OD['destination'][i]
  points = np.concatenate((np.mean([segments_points[o_id-1]['points'][:]],axis=1),np.mean([segments_points[d_id-1]['points'][:]],axis=1)))
  n =  inbound_OD['demand'][i]
  folium.PolyLine(points,clustered_marker = True, color=colormap(n), tooltip=n).add_to(mymap)

mymap.add_child(colormap)

display(mymap)

     direction  origin  destination  demand
0      Inbound       1            1    1065
1      Inbound       1            2     114
2      Inbound       1            3     110
3      Inbound       2            1     284
4      Inbound       2            2     926
...        ...     ...          ...     ...
1248   Stopped      44           44     422
1249   Stopped      44           45      30
1250   Stopped      45           43       1
1251   Stopped      45           44       9
1252   Stopped      45           45     816

[1253 rows x 4 columns]


**Result 2: Trip Attraction and Generations**
* trip Generations map

In [17]:
mymap = folium.Map(width=700,height=550, location=[29.512201, -94.885110], zoom_start=10)

data = OD_data.query('direction != "Stopped"').groupby(['origin']).agg(generation=('origin', np.count_nonzero)).reset_index()

colormap = cm.linear.YlOrRd_03.scale(vmin = data.generation.min(), vmax = data.generation.max())

for i in data.index:
  o_id = data['origin'][i]
  points = segments[o_id-1]['points']
  n =  data['generation'][i]
  folium.Polygon(points, tooltip=n, fill=True, fill_color = colormap(n), fill_opacity=1, stroke= False).add_to(mymap)

mymap.add_child(colormap)

display(mymap)

* trip Attraction map

In [18]:
mymap = folium.Map(width=700,height=550, location=[29.512201, -94.885110], zoom_start=10)

data = OD_data.query('direction != "Stopped"').groupby(['destination']).agg(attraction =('destination', np.count_nonzero)).reset_index()

colormap = cm.linear.YlGnBu_03.scale(vmin = data.destination.min(), vmax = data.destination.max())

for i in data.index:
  o_id = data['destination'][i]
  points = segments[o_id-1]['points']
  n =  data['destination'][i]
  folium.Polygon(points, tooltip=n, fill=True, fill_color = colormap(n), fill_opacity=1, stroke= False).add_to(mymap)

mymap.add_child(colormap)

display(mymap)

**Result 3: Traffic Flow**
* total inbound traffic flow

In [19]:
seg_data = data_df.groupby(['date_interval' , 'hour_interval', 'segment','direction', 'Vessel_Type']).agg(
    flow = ('MMSI', lambda x: x.nunique()), speed =('Speed', lambda x: x.mean())).reset_index()

In [20]:
mymap = folium.Map(width=700,height=550, location=[29.512201, -94.885110], zoom_start=10)

data = seg_data.query('direction=="Inbound"').groupby(['segment']).agg(
    n = ('flow', np.sum)).reset_index()

colormap = cm.linear.YlOrRd_04.scale(vmin = data.n.min(), vmax = data.n.max())

for i in data.index:
  seg_id = data['segment'][i]
  points = segments_points[seg_id-1]['points'][:]
  n =  data['n'][i]
  folium.PolyLine(points,clustered_marker = True, color=colormap(n), tooltip=n).add_to(mymap)

mymap.add_child(colormap)

display(mymap)

* average daily inbound traffic flow

In [21]:
mymap = folium.Map(width=700,height=550, location=[29.512201, -94.885110], zoom_start=10)

data = seg_data.query('direction=="Inbound"').groupby(['segment']).agg(
    n = ('flow',lambda x: x.mean())).reset_index()

colormap = cm.linear.YlOrRd_04.scale(vmin = data.n.min(), vmax = data.n.max())

for i in data.index:
  seg_id = data['segment'][i]
  points = segments_points[seg_id-1]['points'][:]
  n =  data['n'][i]
  folium.PolyLine(points,clustered_marker = True, color=colormap(n), tooltip=n).add_to(mymap)

mymap.add_child(colormap)

display(mymap)

**Result 4: Travel Speed**
* average inbound traffic speed

In [22]:
mymap = folium.Map(width=700,height=550, location=[29.512201, -94.885110], zoom_start=10)

data = seg_data.query('direction=="Inbound"').groupby(['segment']).agg(
    n = ('speed',lambda x: x.mean())).reset_index()

colormap = cm.linear.RdBu_04.scale(vmin = data.n.min(), vmax = data.n.max())

for i in data.index:
  seg_id = data['segment'][i]
  points = segments_points[seg_id-1]['points'][:]
  n =  data['n'][i]
  folium.PolyLine(points,clustered_marker = True, color=colormap(n), tooltip=n).add_to(mymap)

mymap.add_child(colormap)

display(mymap)

**Result 5: Dwell time**
* total dwelltime for stopped vessels

In [34]:
mymap = folium.Map(width=700,height=550, location=[29.512201, -94.885110], zoom_start=10)


data = OD_data.query('direction == "Stopped" & origin==destination').groupby(['origin']).agg(
    dwell_time =('travel_time', np.sum)).reset_index()

colormap = cm.linear.YlOrRd_04.scale(vmin = data.dwell_time.min(), vmax = data.dwell_time.max())

for i in data.index:
  seg_id = data['origin'][i]
  points = np.mean(segments_points[seg_id-1]['points'][:], axis=0).tolist()
  n =  data['dwell_time'][i]
  folium.Circle(points,radius= np.sqrt(n/10), fill=True, color=colormap(n), fill_color=colormap(n), tooltip="dwell time: {}".format(n), opacity = 0.9).add_to(mymap)

mymap.add_child(colormap)

display(mymap)